# Efficient representation of multidimensional arrays.
### Last modification (13.05.2018)


In this tutorial we provide a theoretical backgound on efficient representation of multidimensional arrays and show how these data structures are integrated into [hottbox](https://github.com/hottbox/hottbox) through **TensorCPD**, **TensorTKD** and **TensorTT** classes.

More details on **TensorCPD**, **TensorTKD** and **TensorTT** classes can be found on the [documentation page](https://hottbox.github.io/stable/api/hottbox.core.html#module-hottbox.core).

> **Note:** this tutorial assumes that you are familiar with the basics of tensor algebra and the corresponding conventional notation.  If you are new to this area, the required background is covered in our [introductory notebook](https://github.com/hottbox/hottbox-tutorials/blob/master/1_N-dimensional_arrays_and_Tensor_class.ipynb).

**Requirements:** ``hottbox>=0.1.2``

**Author:** Ilya Kisil - ilyakisil@gmail.com

In [1]:
import numpy as np
from hottbox.core import Tensor, TensorCPD, TensorTKD, TensorTT

![storage_complexity](./images/storage_complexity.png)

# Outer product, rank-1 tensor and definitions of rank of a multi-dimensional array.

## Outer product
The central operator in tensor analysis is the outer product (sometimes refered to as the tensor product). 
Consider tensors $\mathbf{\underline{A}} \in \mathbb{R}^{I_1 \times \cdots \times I_N}$ and $\mathbf{\underline{B}}  \in \mathbb{R}^{J_1 \times \cdots \times J_M}$, then  their outer product yeilds a tensor of higher order then both of them:

$$
\begin{equation}
\begin{aligned}
    \mathbf{\underline{A}} \circ \mathbf{\underline{B}} &= \mathbf{\underline{C}} \in \mathbb{R}^{I_1 \times \cdots \times I_N \times J_1 \times \cdots \times J_M} \\
    a_{i_1,\dots,i_N}b_{j_1,\dots,j_M} &= c_{i_1,\dots,i_N,j_1,\dots,j_M} 
\end{aligned}    
\end{equation}
$$

Most of the time we deal with the outer product of vectors, which significanlty simplifies the general form expressed above and establishes one the of the most fundamenatal definitions.
A tensor of order $N$ is said to be of **rank-1** if it can be represented as an outer product of $N$ vectors. The figure below illustrates an example of rank-1 tensor $\mathbf{\underline{X}}$ and provides intuition of how operation of outer product is computed:

![outerproduct](./images/outerproduct_3.png)

There are several forms of the rank of N-dimensional arrays each of which is accosiated with a representation of a tensor in a particular form:

1. Kryskal rank $\rightarrow$ canonical polyadic form.

- Multi-linear rank $\rightarrow$ tucker form.

- TT rank $\rightarrow$ tensor train form.

> **Note:** The Kryskal rank and the rank of tensor are often time used interchangeably

Each of this representations has the correposing class: **``TensorCPD``**, **``TensorTKD``**,
**``TensorTT``**. All of them come with almost identical API except of obejct creation and,
as a result, the names for some attributes. But before, we can proceed, it is crucial to 
get acquainted with the following definitions.


## Kryskal rank

The rank of a tensor $\mathbf{\underline{X}}$ is defined as the smallest number of rank-one tensors that generate $\mathbf{\underline{X}}$ as their linear combination. 
The figure below illustrates a tensor $\mathbf{\underline{X}}$ of rank $R$.

![cpd_as_rank_one](./images/cpd_as_rank_one.png)


## Multi-linear rank

The multi-linear rank of a tensor $\mathbf{\underline{X}} \in \mathbb{R}^{I_1 \times \cdots \times I_N}$ is the 
$N$-tuple $(R_1, \dots, R_N)$ where each $R_n$ is the rank of the subspace spanned by mode-$n$ 
fibers, i.e. $R_n = \text{rank} \big( \mathbf{X}_{(n)} \big)$.

> **Note:** for a tensor of order $N$ the values $R_1, R_2, \dots , R_N$ are not necessarily the
same, whereas, for matrices (tensors of order 2) the equality $R_1 = R_2$ always holds, where 
$R_1$ and $R_2$ are the matrix column rank and row rank respectively.








# Canonical polydiac form and **TensorCPD** class

![tensorcpd](./images/TensorCPD.png)

The canonical polyadic (CP) form represents a tensor as linear combination of rank-1 tensors. For a third order tensor or rank $R$ it can be expressed as follows:

$$\mathbf{\underline{X}} = \sum_{r=1}^R \lambda_{r} \cdot \mathbf{a}_r \circ \mathbf{b}_r \circ \mathbf{c}_r$$

The vectors $\mathbf{a}_r, \mathbf{b}_r$ and $\mathbf{c}_r$ are oftentime combined into corresponding **factor matrices**:

$$
\mathbf{A} = \Big[ \mathbf{a}_1 \cdots \mathbf{a}_R \Big] \quad
\mathbf{B} = \Big[ \mathbf{b}_1 \cdots \mathbf{b}_R \Big] \quad
\mathbf{C} = \Big[ \mathbf{c}_1 \cdots \mathbf{c}_R \Big] \quad
$$

Thus, if we employ the mode-$n$ product, the canonical polyadic representation takes form:

$$
\mathbf{\underline{X}} = \mathbf{\underline{\Lambda}} \times_1 \mathbf{A} \times_2 \mathbf{B} \times_3 \mathbf{C} = \Big[\mathbf{\underline{\Lambda}}; \mathbf{A}, \mathbf{B}, \mathbf{C} \Big]
$$
where the elements on the super-diagonal of $\mathbf{\underline{\Lambda}}$ are occupied by the values
$\lambda_r$ and all other equal to zero.

In **``hottbox``** this form is available through the **``TensorCPD``** class.
In order to create such object, you need to pass a list of factor matrices (2d ``numpy`` arrays) and a vector of values (as 1d ``numpy`` array) for the main diagonal:

```python
tensor_cpd = TensorCPD(fmat=[A, B, C], core_values=values)
```

> **Note:** all matrices should have the same number of columns and be equal to the length of ``values``


In [2]:
I, J, K = 5, 6, 7  # define shape of the tensor in full form
R = 4  # define Kryskal rank of a tensor in CP form 

A = np.arange(I * R).reshape(I, R)
B = np.arange(J * R).reshape(J, R)
C = np.arange(K * R).reshape(K, R)
values = np.arange(R)
# values = np.random.rand(R)

tensor_cpd = TensorCPD(fmat=[A, B, C], core_values=values)

The list of factor matrices is stored in **`_fmat`** attribute (placeholder) which can (should) be accessed through the correspodning property **`fmat`**.

It is slightly different for the core values.
Even though we passed the values for the super-diagonal and they are stored in **`_core_values`** attribute (placeholder), they cannot (should not) be accessed directly. 
Instead, they are used to create a core tensor of the CP representation when the corresponding property is called:

```python
tensor_cpd.core
```
This returns an object of the **``Tensor``** class with the **``_core_values``** placed on its super-diagonal.

In [3]:
print('Factor matrices')
for mode, fmat in enumerate(tensor_cpd.fmat):
    print('Mode-{} factor matrix is of shape {}'.format(mode, fmat.shape))
    
print('\nCore tensor')
print(type(tensor_cpd.core))
tensor_cpd.core.describe()
tensor_cpd.core.data

Factor matrices
Mode-0 factor matrix is of shape (5, 4)
Mode-1 factor matrix is of shape (6, 4)
Mode-2 factor matrix is of shape (7, 4)

Core tensor
<class 'hottbox.core.structures.Tensor'>
This tensor is of order 3, consists of 64 elements and its Frobenious norm = 3.74.
Sizes and names of its modes are (4, 4, 4) and {0: 'mode-0', 1: 'mode-1', 2: 'mode-2'} respectively.


array([[[ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  2.,  0.],
        [ 0.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  3.]]])

In order to convert **``TensorCPD``** into the full representation, simply call: 

```python
tensor_cpd.reconstruct
```

This return an object of the **``Tensor``** class with N-dimensional array calculated as 
described above and being assinged to the **``_data``** attibute.

In [4]:
tensor_full = tensor_cpd.reconstruct

print(type(tensor_full))
tensor_full.describe()
tensor_full.data

<class 'hottbox.core.structures.Tensor'>
This tensor is of order 3, consists of 210 elements and its Frobenious norm = 238601.50.
Sizes and names of its modes are (5, 6, 7) and {0: 'mode-0', 1: 'mode-1', 2: 'mode-2'} respectively.


array([[[    98.,    242.,    386.,    530.,    674.,    818.,    962.],
        [   242.,    610.,    978.,   1346.,   1714.,   2082.,   2450.],
        [   386.,    978.,   1570.,   2162.,   2754.,   3346.,   3938.],
        [   530.,   1346.,   2162.,   2978.,   3794.,   4610.,   5426.],
        [   674.,   1714.,   2754.,   3794.,   4834.,   5874.,   6914.],
        [   818.,   2082.,   3346.,   4610.,   5874.,   7138.,   8402.]],

       [[   242.,    610.,    978.,   1346.,   1714.,   2082.,   2450.],
        [   610.,   1586.,   2562.,   3538.,   4514.,   5490.,   6466.],
        [   978.,   2562.,   4146.,   5730.,   7314.,   8898.,  10482.],
        [  1346.,   3538.,   5730.,   7922.,  10114.,  12306.,  14498.],
        [  1714.,   4514.,   7314.,  10114.,  12914.,  15714.,  18514.],
        [  2082.,   5490.,   8898.,  12306.,  15714.,  19122.,  22530.]],

       [[   386.,    978.,   1570.,   2162.,   2754.,   3346.,   3938.],
        [   978.,   2562.,   4146.,   5730.,   

# Tucker form and **TensorTKD** class

![tensortkd](./images/TensorTKD.png)

For a tensor $\mathbf{\underline{X}} \in \mathbb{R}^{I \times J \times K}$ illustrated above, the tucker form
represents it as a dense core tensor $\mathbf{\underline{G}}$ with multi-linear rank ($Q, R, P$) and a set of
factor matrices $\mathbf{A} \in \mathbb{R}^{I \times Q}, \mathbf{B} \in \mathbb{R}^{J \times R}$ and $\mathbf{C} \in
\mathbb{R}^{K \times P}$.

The tucker form of a tensor is closely related to the CP form and can be expressed through a 
sequence of mode-$n$ products in a similar way.

$$
\mathbf{\underline{X}} = \mathbf{\underline{G}} \times_1 \mathbf{A} \times_2 \mathbf{B} \times_3 \mathbf{C} = \Big[\mathbf{\underline{G}}; \mathbf{A}, \mathbf{B}, \mathbf{C} \Big]
$$

In **``hottbox``** this form is available through the **``TensorTKD``** class.
In order to create such object, you need to pass a list of factor matrices (2d ``numpy`` arrays) and values for the core tensor (as n-dimensional ``numpy`` array):

```python
tensor_tkd = TensorTKD(fmat=[A, B, C], core_values=values)
```

> **Note:** the number of columns in each of the factor matrices should be the same as the corresponding size of the ``numpy`` array with the values for the core tensor

In [5]:
I, J, K = 5, 6, 7  # define shape of the tensor in full form
Q, R, P = 2, 3, 4  # define multi-linear rank of the tensor in Tucker form

A = np.arange(I * Q).reshape(I, Q)
B = np.arange(J * R).reshape(J, R)
C = np.arange(K * P).reshape(K, P)
values = np.arange(Q * R * P).reshape(Q, R, P)
# values = np.random.rand(Q * R * P).reshape(Q, R, P)

tensor_tkd = TensorTKD(fmat=[A, B, C], core_values=values)

By analogy with the **`TensorCPD`**, the list of factor matrices is stored in **`_fmat`** attribute (placeholder) which can (should) be accessed through the correspodning property **`fmat`**.

Similarly, the values of the core tensor are stored in **`_core_values`** attribute (placeholder) and they cannot (should not) be accessed directly.
Instead, they are used to create an obecjt of **`Tensor`** class, when the corresponding property is called:

```python
tensor_tkd.core
```

> **Note:** the core values occupy all data values of a core tensor, as opposed to **`TensorCPD`** class where they are placed on the main diagonal.

In [6]:
print('Factor matrices')
for mode, fmat in enumerate(tensor_tkd.fmat):
    print('Mode-{} factor matrix is of shape {}'.format(mode, fmat.shape))
    
print('\nCore tensor')
print(type(tensor_tkd.core))
tensor_tkd.core.describe()
tensor_tkd.core.data

Factor matrices
Mode-0 factor matrix is of shape (5, 2)
Mode-1 factor matrix is of shape (6, 3)
Mode-2 factor matrix is of shape (7, 4)

Core tensor
<class 'hottbox.core.structures.Tensor'>
This tensor is of order 3, consists of 24 elements and its Frobenious norm = 65.76.
Sizes and names of its modes are (2, 3, 4) and {0: 'mode-0', 1: 'mode-1', 2: 'mode-2'} respectively.


array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]],

       [[12, 13, 14, 15],
        [16, 17, 18, 19],
        [20, 21, 22, 23]]])

In order to convert **``TensorTKD``** into the full representation, simply call: 

```python
tensor_tkd.reconstruct
```

This return an object of the **``Tensor``** class with N-dimensional array calculated as 
described above and being assinged to the **``_data``** attibute.

In [7]:
tensor_full = tensor_tkd.reconstruct

print(type(tensor_full))
tensor_full.describe()
tensor_full.data

<class 'hottbox.core.structures.Tensor'>
This tensor is of order 3, consists of 210 elements and its Frobenious norm = 3535670.11.
Sizes and names of its modes are (5, 6, 7) and {0: 'mode-0', 1: 'mode-1', 2: 'mode-2'} respectively.


array([[[    378,    1346,    2314,    3282,    4250,    5218,    6186],
        [   1368,    4856,    8344,   11832,   15320,   18808,   22296],
        [   2358,    8366,   14374,   20382,   26390,   32398,   38406],
        [   3348,   11876,   20404,   28932,   37460,   45988,   54516],
        [   4338,   15386,   26434,   37482,   48530,   59578,   70626],
        [   5328,   18896,   32464,   46032,   59600,   73168,   86736]],

       [[   1458,    5146,    8834,   12522,   16210,   19898,   23586],
        [   5112,   17944,   30776,   43608,   56440,   69272,   82104],
        [   8766,   30742,   52718,   74694,   96670,  118646,  140622],
        [  12420,   43540,   74660,  105780,  136900,  168020,  199140],
        [  16074,   56338,   96602,  136866,  177130,  217394,  257658],
        [  19728,   69136,  118544,  167952,  217360,  266768,  316176]],

       [[   2538,    8946,   15354,   21762,   28170,   34578,   40986],
        [   8856,   31032,   53208,   75384,   

# Tensor train form and **TensorTT** class

![tensortt](./images/TensorTT.png)

Tensor trains (TTs) are the simplest kinds of tensor networks, i.e. a decomposition of a high-order tensor in a set of sparsely interconnected lower-order tensors and factor matrices. Mathematically, an $N$-th order tensor $\mathbf{\underline{X}} \in \mathbb{R}^{I_1 \times I_2 \times \cdots \times I_N}$ can be expressed as a TT as

\begin{equation}
\mathbf{\underline{X}} = \mathbf{A} \times^1_2 \mathbf{\underline{G}}^{(1)}  \times^1_3 \mathbf{\underline{G}}^{(2)}   \times^1_3 \cdots \times^1_3 \mathbf{\underline{G}}^{(N-1)} \times^1_3 \mathbf{B} = \Big[  \mathbf{A}, \mathbf{\underline{G}}^{(1)}, \mathbf{\underline{G}}^{(2)}, \cdots, \mathbf{\underline{G}}^{(N-1)}, \mathbf{B}  \Big]
\end{equation}

Each element of a TT is generally referred to as $\textit{core}$, and $\mathbf{A} \in \mathbb{R}^{I_1 \times R_1}$, $\mathbf{B} \in \mathbb{R}^{R_{N-1}\times I_N}$, $\mathbf{\underline{G}}^{(n)} \in \mathbb{R}^{R_n \times I_{n+1} \times R_{n+1}}$


In **``hottbox``** this form is available through the **``TensorTT``** class.
In order to create such object, you need to pass a list of values (as ``numpy`` arrays) for 
cores the shape of a tensor in full form (as ``tuple``):

```python
tensor_tt = TensorTT(core_values=values, ft_shape=shape)
```

> **Note:** it is very likely that the argument ``ft_shape`` will be removed soon. Check
the [CHANGELOG](https://github.com/hottbox/hottbox/blob/master/CHANGELOG.md) of **``hottbox``**

## TT-rank
Given a TT decomposition of an $N$-th order tensor $\mathbf{\underline{X}} \in \mathbb{R}^{I_1 \times I_2 \times \cdots \times I_N}$, as $\mathbf{\underline{X}} = \Big[  \mathbf{A}, \mathbf{\underline{G}}^{(1)}, \mathbf{\underline{G}}^{(2)}, \cdots, \mathbf{\underline{G}}^{(N-1)}, \mathbf{B}  \Big]$, with cores $\mathbf{\underline{G}}^{(n)} \in \mathbb{R}^{R_n \times I_{n+1} \times R_{n+1}}$, the tuple $(R_1, R_2, \dots, R_{N-1})$ is called the TT-rank.

In [8]:
I, J, K = 5, 6, 7  # define shape of the tensor in full form
shape =  (I, J, K)
R1, R2 = 2, 3  # define tt rank of the tensor in Tensor train form

values_1 = np.arange(I * R1).reshape(I, R1)
values_2 = np.arange(R1 * J * R2).reshape(R1, J, R2)
values_3 = np.arange(R2 * K).reshape(R2, K)
values = [values_1, values_2, values_3]

tensor_tt = TensorTT(core_values=values, ft_shape=shape)

The list of core tensors is stored in **`_core_values`** attribute (placeholder) which can (should), but they cannot (should not) be accessed directly. Instead, they can be accessed all together via **`cores`** property which outputs a list where each entry from **`_core_values`** is converted to a **`Tensor`** class:

```python
tensor_tt.cores
```

> **Note:** All components of the Tensor Train representation are conventionally considered to be a core therefore, even matrices are objects of **`Tensor`** class.

In [9]:
for i, tt_core in enumerate(tensor_tt.cores):        
    print('\n\tCore tensor #{} of TT representation'.format(i))    
    print(type(tt_core))
    tt_core.describe()
    print(tt_core.data)


	Core tensor #0 of TT representation
<class 'hottbox.core.structures.Tensor'>
This tensor is of order 2, consists of 10 elements and its Frobenious norm = 16.88.
Sizes and names of its modes are (5, 2) and {0: 'mode-0', 1: 'mode-1'} respectively.
[[0 1]
 [2 3]
 [4 5]
 [6 7]
 [8 9]]

	Core tensor #1 of TT representation
<class 'hottbox.core.structures.Tensor'>
This tensor is of order 3, consists of 36 elements and its Frobenious norm = 122.11.
Sizes and names of its modes are (2, 6, 3) and {0: 'mode-0', 1: 'mode-1', 2: 'mode-2'} respectively.
[[[ 0  1  2]
  [ 3  4  5]
  [ 6  7  8]
  [ 9 10 11]
  [12 13 14]
  [15 16 17]]

 [[18 19 20]
  [21 22 23]
  [24 25 26]
  [27 28 29]
  [30 31 32]
  [33 34 35]]]

	Core tensor #2 of TT representation
<class 'hottbox.core.structures.Tensor'>
This tensor is of order 2, consists of 21 elements and its Frobenious norm = 53.57.
Sizes and names of its modes are (3, 7) and {0: 'mode-0', 1: 'mode-1'} respectively.
[[ 0  1  2  3  4  5  6]
 [ 7  8  9 10 11 12

If you what to access a specific core tensor of the TT representation, then it is more efficient to use a corresponding method which takes a positional number of desired core as input parameters

```python
tensor_tt.core(i=0)
```

> **Note:** this parameter should not exceed the order of TT representation

In [10]:
for i in range(tensor_tt.order):
    tt_core = tensor_tt.core(i)
    print('\n\tCore tensor #{} of TT representation'.format(i))    
    print(type(tt_core))
    tt_core.describe()
    print(tt_core.data)


	Core tensor #0 of TT representation
<class 'hottbox.core.structures.Tensor'>
This tensor is of order 2, consists of 10 elements and its Frobenious norm = 16.88.
Sizes and names of its modes are (5, 2) and {0: 'mode-0', 1: 'mode-1'} respectively.
[[0 1]
 [2 3]
 [4 5]
 [6 7]
 [8 9]]

	Core tensor #1 of TT representation
<class 'hottbox.core.structures.Tensor'>
This tensor is of order 3, consists of 36 elements and its Frobenious norm = 122.11.
Sizes and names of its modes are (2, 6, 3) and {0: 'mode-0', 1: 'mode-1', 2: 'mode-2'} respectively.
[[[ 0  1  2]
  [ 3  4  5]
  [ 6  7  8]
  [ 9 10 11]
  [12 13 14]
  [15 16 17]]

 [[18 19 20]
  [21 22 23]
  [24 25 26]
  [27 28 29]
  [30 31 32]
  [33 34 35]]]

	Core tensor #2 of TT representation
<class 'hottbox.core.structures.Tensor'>
This tensor is of order 2, consists of 21 elements and its Frobenious norm = 53.57.
Sizes and names of its modes are (3, 7) and {0: 'mode-0', 1: 'mode-1'} respectively.
[[ 0  1  2  3  4  5  6]
 [ 7  8  9 10 11 12

In order to convert **``TensorTT``** into the full representation, simply call: 

```python
tensor_tt.reconstruct
```

This return an object of the **``Tensor``** class with N-dimensional array calculated as 
described above and being assinged to the **``_data``** attibute.

In [11]:
tensor_full = tensor_tt.reconstruct

print(type(tensor_full))
tensor_full.describe()
tensor_full.data

<class 'hottbox.core.structures.Tensor'>
This tensor is of order 3, consists of 210 elements and its Frobenious norm = 91247.96.
Sizes and names of its modes are (5, 6, 7) and {0: 'mode-0', 1: 'mode-1', 2: 'mode-2'} respectively.


array([[[  413,   470,   527,   584,   641,   698,   755],
        [  476,   542,   608,   674,   740,   806,   872],
        [  539,   614,   689,   764,   839,   914,   989],
        [  602,   686,   770,   854,   938,  1022,  1106],
        [  665,   758,   851,   944,  1037,  1130,  1223],
        [  728,   830,   932,  1034,  1136,  1238,  1340]],

       [[ 1309,  1486,  1663,  1840,  2017,  2194,  2371],
        [ 1624,  1846,  2068,  2290,  2512,  2734,  2956],
        [ 1939,  2206,  2473,  2740,  3007,  3274,  3541],
        [ 2254,  2566,  2878,  3190,  3502,  3814,  4126],
        [ 2569,  2926,  3283,  3640,  3997,  4354,  4711],
        [ 2884,  3286,  3688,  4090,  4492,  4894,  5296]],

       [[ 2205,  2502,  2799,  3096,  3393,  3690,  3987],
        [ 2772,  3150,  3528,  3906,  4284,  4662,  5040],
        [ 3339,  3798,  4257,  4716,  5175,  5634,  6093],
        [ 3906,  4446,  4986,  5526,  6066,  6606,  7146],
        [ 4473,  5094,  5715,  6336,  6957,  7578,  

## Additional notes on API of **Tensor** class

1. In order to create a duplicate of an object of **`TensorCPD`**, **`TensorTKD`** and **`TensorTT`**  classes then use method **`copy()`**